In [1]:
import numpy as np
import numpy.typing as npt
# from numba import jit, njit, prange
import pickle
from dataclasses import dataclass, asdict, field
import argparse
import hashlib
import json
from os import listdir
from os.path import isfile, join
# import pandas as pd
from pathlib import Path
from typing import Any
import time
import sys
import copy
import concurrent.futures as cf

from src.dataclass import (
    Input, Lattice, Parameter, Train, Save,
    Processed_Input, Topology, Conjugate, Result, summarize_results
)
from src.manage_data import save_result, save_log, load_result
from src.initial_state import get_initial_state
from src.process_input import (get_processed_input, get_T_and_H, get_J)
from src.metropolis import execute_metropolis_update
from src.function import time_correlation
from src.process_output import get_result

In [2]:
def run_ensemble(
    input: Input,
    processed_input: Processed_Input,
    # J: npt.NDArray,
    ensemble_num: int
) -> tuple[int, Result]:
# tuple[int, tuple, npt.NDArray[np.float64], int]

    (size, dimension, iteration, sweep,
     measurement, interval, recent, threshold) = (
        input.lattice.size,
        input.lattice.dimension,
        input.train.iteration,
        input.train.sweep,
        input.train.measurement,
        input.train.interval,
        input.train.recent,
        input.train.threshold,
    )

    begin = time.perf_counter()

    J = get_J(input, processed_input) # get coupling parameter

    initial = get_initial_state(input) # initialize state
    update = initial.copy()
    autocorr, autocorr_len = np.empty(iteration+1, dtype=np.float64), 1 # autocorrelation
    autocorr[0] = time_correlation(initial, initial, size**dimension)

    now = time.perf_counter()
    # Removing initial state effect until autocorrelation satsisfies certain criteria
    for _ in range(iteration):
        update = execute_metropolis_update(
            input, processed_input, J, update)
        autocorr[autocorr_len] = np.abs(time_correlation(
            update, initial, size**dimension))
        autocorr_len += 1
        if autocorr_len > recent:
            temp = autocorr[autocorr_len-recent:autocorr_len]
            if np.average(temp) < threshold or np.std(temp) < threshold:
                break
    
    # if(ensemble_num==1):  
    print(f"initial effect removed, iter: {autocorr_len}, time: {time.perf_counter()-now}s, avg: {np.average(autocorr[autocorr_len-recent:autocorr_len])}, std: {np.std(autocorr[autocorr_len-recent:autocorr_len])}")

    now = time.perf_counter()
    # Collect raw output after performing metropolis update
    raw_output = np.empty((measurement, size**dimension), dtype=np.complex128)
    for i in range(sweep):
        update = execute_metropolis_update(input, processed_input, J, update)
        if autocorr_len <= iteration:
            autocorr[autocorr_len] = np.abs(
                time_correlation(update, initial, size**dimension))
            autocorr_len += 1
        if i % interval == interval - 1:
            raw_output[int(i/interval)] = update.copy() #! .copy() maybe not essential?
    
    # if(ensemble_num==1):
    print(f"raw output collected, iter: {sweep}, time: {time.perf_counter()-now}s")

    now = time.perf_counter()
    result = get_result(input, processed_input, raw_output, J) # process raw output
    if(ensemble_num==1):  print(f"raw output processed, time: {time.perf_counter()-now}s")
    result.autocorrelation = np.array(autocorr)
    result.time = time.perf_counter() - begin

    return ensemble_num, result

In [3]:
def sampling(
    input: Input,
    processed_input: Processed_Input,
) -> None:

    max_workers, ensemble = (
        input.train.max_workers,
        input.train.ensemble,
    )
    
    with cf.ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(run_ensemble, input, processed_input, i + 1)
            for i in range(ensemble) # execute each ensemble
        ]

        ensemble_results : list[Result] = []        
        finished = 0
        
        for future in cf.as_completed(futures):
            finished += 1
            ensemble_num, single_result = future.result()
            ensemble_results.append(single_result)
            
            print(f"{ensemble_num}: {single_result}")
    
    result = summarize_results(ensemble_results)

    save_log(input, result)

    if input.save.save:
        save_result(input, result)

    print(input)
    print(result, "\n")

In [4]:
def experiment(args: argparse.Namespace) -> None:
    lattice = Lattice(
        args.state, args.size, args.dimension,
        args.ghost, args.initial)
    parameter = Parameter(
        args.Tc, args.Hc, args.Jm, args.Jv, args.mode,
        args.variable, args.multiply, args.base, args.exponent)
    train = Train(
        args.iteration, args.sweep, args.measurement, args.interval,
        args.ensemble, args.max_workers, args.threshold, args.recent)
    save = Save(args.environment, args.location, args.save)

    input = Input(lattice, parameter, train, save)
    input.parameter.T, input.parameter.H = args.T, args.H

    now = time.perf_counter()
    processed_input = get_processed_input(input) # process input
    print(f"input processed, time: {time.perf_counter()-now}s")
    
    input.parameter.T, input.parameter.H = get_T_and_H(input) # get temperature and external field

    print(input, "\n")
    sampling(input, processed_input)

In [5]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

"""
Lattice Condition
"""
args.state = 3
args.size = 8
args.dimension = 3
args.ghost = 0
args.initial = "uniform" # "uniform", "random"

"""
Parameter Condition
"""
args.T, args.H = 1.0, 0.0
# q=2: 2.2692, q=3: 1.4925
# args.Tc = 1/((1-1/args.state)*np.log(1+np.sqrt(args.state)))
args.Tc = 2.0
args.Hc = 0.0
args.Jm = 1.0
args.Jv = 0.0

args.mode = "normal"  # 'normal', 'critical' and 'manual'
args.variable = "T"  # 'T', 'H'
args.multiply = 0.1**4
args.base = 2.0
args.exponent = 13

"""
Train Condition
"""
args.iteration = 2**10
args.measurement = 2**14 # total number of measurement
args.interval = 2**0 # measure once in every "interval" iteration
args.sweep = args.measurement * args.interval # total number of iteration
args.max_workers = 8
args.ensemble = args.max_workers * 2**6
args.threshold = 0.1**2
args.recent = 10**2

"""
Save Condition
"""
args.environment = "local" # "server" or "local"
args.location = "temp"
args.save = True  # True or False

# experiment(args)

name1, list1 = 'exponent', [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13]

for var1 in list1:
    setattr(args, name1, var1)
    experiment(args)

input processed, time: 3.2311878359978436s
Input(lattice=Lattice(state=3, size=64, dimension=2, ghost=0, initial='uniform'), parameter=Parameter(T=1.5008, H=0.0, Tc=1.5, Hc=0.0, Jm=1.0, Jv=0.0, mode='normal', variable='T', multiply=0.00010000000000000002, base=2.0, exponent=3), train=Train(iteration=1024, sweep=16384, measurement=16384, interval=1, ensemble=24, max_workers=3, threshold=0.010000000000000002, recent=100), save=Save(environment='local', location='temp', save=True)) 

initial effect removed, iter: 1025, time: 1.909536917002697s, avg: 0.60009765625, std: 0.037918025855271625
initial effect removed, iter: 1025, time: 1.9297781499990379s, avg: 0.690921630859375, std: 0.027246954873535952
initial effect removed, iter: 1025, time: 1.967803437000839s, avg: 0.630174560546875, std: 0.028326867206632092
raw output collected, iter: 16384, time: 13.964500553003745s
order parameter processed, 0.10935910400439752s
raw output collected, iter: 16384, time: 14.342910845000006s
order param